# Patient Appointment No-Show Predictor (Complete Code)

In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import joblib

In [2]:
# Step 2: Load Dataset
df = pd.read_csv('appointments.csv')

In [3]:
# Step 3: Data Preprocessing
# Convert dates to datetime
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [4]:
# Create waiting time feature
df['WaitingDays'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

In [5]:
# Drop irrelevant columns
df.drop(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], axis=1, inplace=True)

In [6]:
# Map target column
df['No-show'] = df['No-show'].map({'Yes': 1, 'No': 0})

In [7]:
# Filter out invalid ages
df = df[df['Age'] >= 0]

In [8]:
# Separate features and target
X = df.drop('No-show', axis=1)
y = df['No-show']

In [9]:
# Identify categorical and numerical columns
categorical_cols = ['Gender', 'Neighbourhood', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'SMS_received']
numerical_cols = ['Age', 'WaitingDays']

In [10]:
# Step 4: Pipeline for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ], remainder='passthrough')

In [11]:
# Step 5: Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [12]:
# Apply preprocessing
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [13]:
# Step 6: Handle class imbalance
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_encoded, y_train)

In [14]:
# Step 7: Train Model
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train_bal, y_train_bal)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [15]:
# Step 8: Evaluate
y_pred = model.predict(X_test_encoded)
print("F1 Score:", f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

F1 Score: 0.30610932475884245
[[15521  2121]
 [ 3274  1190]]
              precision    recall  f1-score   support

           0       0.83      0.88      0.85     17642
           1       0.36      0.27      0.31      4464

    accuracy                           0.76     22106
   macro avg       0.59      0.57      0.58     22106
weighted avg       0.73      0.76      0.74     22106



In [16]:
# Step 9: Save Model and Preprocessor
joblib.dump(model, 'rf_model.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']